In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
df_reviews = spark.read.csv('googleplaystore_user_reviews.csv',inferSchema=True, header =True)
df_play = spark.read.csv('googleplaystore.csv',inferSchema=True, header =True)

In [ ]:
data = spark.read.csv('googleplaystore.csv',inferSchema=True, header=True)


In [ ]:
df_play.printSchema()

In [ ]:
## Need to do some processing to make sense of this data

spark.sql("select Category, AVG(Rating) from plays group by 1 order by 2").show()



In [ ]:
def size_clean(item):
  if 'k' == item[-1].lower():
    new_item = float(item[0:-2])*10*10*10
  elif 'm' == item[-1].lower():
    new_item = float(item[0:-2])*10*10*10*10*10*10
  else:
    try:
      new_item = float(item[0:-2])
    except:
      new_item = 0
  return new_item

spark.udf.register("size_clean", size_clean)

def price_clean(item):
  try:
    new_price = float(item[1:])
  except:
    new_price = 0
  return new_price

spark.udf.register("price_clean", price_clean)

In [ ]:
sql = """select App,
          Category,
          Reviews,
          size_clean(Size),
          Installs,
          price_clean(Price) as Price,
          `Content Rating`as Content_Rating,
          cast(Rating as double) as Rating
    from plays
    where 1=1
    and Category != "1.9"
    and Rating is not null
    and size_clean(Size) != 0
    order by Price desc
    """

spark.sql(sql).show()